# Preâmbulo

Imports básicos


In [0]:
# Basic imports.
import os
import csv
import time
import random
import pandas as pd
import numpy as np
import torch

from torch import nn
from torch import optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from sklearn import metrics
from sklearn.model_selection import train_test_split

from torchvision import models

from torchtext import data
from torchtext import datasets

import spacy
! python -m spacy download en
! python -m spacy download fr

from matplotlib import pyplot as plt
%matplotlib inline

cudnn.benchmark = True

SEED = 1234
torch.manual_seed(SEED)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/fr
You can now load the model via spacy.load('fr')


In [0]:
# Setting predefined arguments.
args = {
    'epoch_num': 100,       # Number of epochs.
    'lr': 1e-3,           # Learning rate.
    'weight_decay': 5e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 6,     # Number of workers on data loader.
    'batch_size': 10,     # Mini-batch size.
    'max_length': 50,    # Maximun length of predicted sentence
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

cuda


# Generating Sequences


Dentre os tipos de problemas solucionáveis com modelos recorrentes, dois deles são baseados em geração de sequências: Os problemas One-to-Many,  e os Many-to-Many não sincronizados. <br>

Tipicamente os modelos de geração de sequências são baseados em arquiteturas **Encoder-Decoder**, onde a entrada é codificada para uma forma fixa, e então decodificada passo a passo em uma sequência.

<img src="http://karpathy.github.io/assets/rnn/diags.jpeg" width="600">


## One-to-Many: Image Captioning

O problema de legendar (ou descrever) imagens, se encaixa na categoria One-to-Many, pois cada imagem é considerada uma unidade atômica, ou seja, não é modelada como uma sequência. Já a saída é uma sequência de caracteres semanticamente relacionados com a imagem. Na figura a seguir vemos uma representação superficial do problema.

<img src="https://drive.google.com/uc?export=view&id=10YhOB7pvnhXUhqu08JNJ-YcgoyPp8c3K" height="350">

Em termos de modelagem de solução, temos que a imagem deve ser mapeada para um espaço latente que provê um vetor de características de contexto (**context feature**).  Essa etapa consiste na codificação da sua entrada (**encoder**), destacando características semanticamente relevantes para a etapa de decodificação (**decoder**).

Sendo a saída uma sequência de palavras, é comum que o decoder seja composto por camadas recorrentes, que na primeira iteração recebem como entrada:
*  Hidden state inicial ($h_0$): context feature, ou seja, a saída produzida pelo encoder
*  Input inicial ($x_0$): Token especial de início de sequência (**```<sos>```** - start of sequence)

As iterações seguintes recebem como entrada os resultados produzidos na iteração anterior, ou seja:
*  Hidden state inicial ($h_t$): $h_{t-1}$
*  Input inicial ($x_t$): $y_{t-1}$

O fim das iterações é determinado pela geração do token **```<eos>```** indicando o término da sequência (end of sequence). A figura a seguir apresenta uma representação visual desse pipeline. Em azul é apresentada a entrada única do decoder (```<sos>```) e em vermelho as saídas do modelo.<br><br>

<img src="https://drive.google.com/uc?export=view&id=1j7SUTfGIHi7XIPv8YRKF1XLTTmJfvQrd" width="750"><br><br>


O código a seguir apresenta uma ilustração de pequeno porte de um modelo de Image Captioning, sem adição de transformações ou modelos de atenção. Para problemas do mundo real, aplicar o conceito de Atenção melhora significativemente a qualidade de modelos, mas a princípio vamos concentrar na arquitetura recorrente Encoder-Decoder. <br>
Para quem quiser saber mais, recomendo esse tutorial de Image Captioning: https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Image-Captioning

Perceba no código a seguir a implementação de dois tipos de ```forward()```:
* ```forward_inference()```: A forma mais direta de decodificar a entrada em uma sequência de tokens, usando a saída do timestep anterior como entrada do timestep seguinte. Requer a implementação de um **loop explícito**, diferente do forward encapsulado que vínhamos utilizando.
* ```forward()```: Em tempo de treino, é possível realizar o forward encapsulado, alimentando a sequência target como entrada do modelo recorrente. Nesse caso o comprimento das sequências é conhecido, e o loop encapsulado pode ser interrompido ao final da sequência target. **Na prática recomenda-se fazer parte do treinamento com os targets e outra parte sem**. A figura a seguir apresenta a diferença sutil que permite o encapsulamento do forward de batches. Novamente em azul são apresentadas as entradas fornecidas ao modelo, e em vermelho as saídas. <br><br>

<img src="https://drive.google.com/uc?export=view&id=1XBnnnbAWx2y63JiZ9G8zBtxxll0YpV_F" width="750"> <br><br>

Vale explicitar que **cada iteração no decoder é composta pelas camadas de embedding, RNN e Linear**. Em problemas que precisam fazer classifcação em múltiplas iterações, usa-se de um artifício para permitir o forward encapsulado. Dada uma saída recorrente com shape ```(seq_len, batch_size, hidden_size)```, o forward na camada linear é realizado redimensionando a saída para a forma  ```(seq_len * batch_size, hidden_size)```, de modo que a camada linear interprete como um grande batch de características. Como a dependência temporal é modelada internamente pela GRU, as amostras podem ser alimentadas individualmente para a camada Linear.

```python
outputs_rnn, hidden = self.gru(inputs, hidden)

input_linear = output_rnn.view(output_rnn.size(0)*output_rnn.size(1), output_rnn.size(2))
output = self.softmax(self.out(input_linear))

```

<img src="https://drive.google.com/uc?export=view&id=1eFDgd6gn_l-lcKcQIuQA7ooI_ZkvWYrp" width="500">

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, output_size, max_length, dropout_p=0.1):
        super(DecoderRNN, self).__init__()
        
        self.embed_size  = embed_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.dropout_p = dropout_p
        self.max_length = max_length

        # Word embedding
        self.embedding = nn.Embedding(self.output_size, self.embed_size)
        self.dropout = nn.Dropout(self.dropout_p)
        
        # Recurrent feature
        self.gru = nn.GRU(self.embed_size, self.hidden_size)
        
        # Classify next word
        self.out = nn.Linear(self.hidden_size, self.output_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, hidden, target=None, lengths=None):

        if target is None:
          return self.forward_inference(hidden)
      
        embedded = self.dropout(self.embedding(target))

        packed_inputs = nn.utils.rnn.pack_padded_sequence(embedded, lengths)        
        packed_outputs, hidden = self.gru(packed_inputs, hidden)
        output_rnn, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_outputs)
        
        output = output_rnn.view(output_rnn.size(0)*output_rnn.size(1), output_rnn.size(2))
        output = self.softmax(self.out(output))
        output = output.view(output_rnn.size(0), output_rnn.size(1), -1)
        
        return output

      
    def forward_inference(self, hidden):
      
        # Start inference with <sos> token
        input = torch.tensor(TEXT.vocab.stoi["<sos>"]).to(args['device'])
        
        outputs = []
        # Iterate to a maximum length
        for i in range(self.max_length):
          
          # Forward single sample
          embedded = self.embedding(input).view(1,1,-1)
          output_rnn, hidden = self.gru(embedded, hidden)
          output = self.softmax(self.out(output_rnn[0]))
          
          outputs.append(output.detach())
          
          # Current output feeds future input
          topv, topi = output.topk(1)
          input = topi.squeeze().detach()  

          # Finish inference when <eos> generated
          if input == TEXT.vocab.stoi["<eos>"]:
            break
            
        # Return sequence of tokens produced 
        # Either interrupted by producing <eos>
        # Or interrupted by max_length
        return torch.stack(outputs)
          
        
      

####### Build encoder ###########
# encoder = models.resnet18(pretrained=True)
encoder = models.resnet18()
num_features = encoder.fc.in_features

# Remove linear layers
modules = list(encoder.children())[:-2]
encoder = nn.Sequential(*modules)
#################################


####### Build decoder ###########
vocab_size   = len(TEXT.vocab)
embed_size   = 100
hidden_size  = num_features
max_length   = args['max_length'] 

decoder = DecoderRNN(embed_size, hidden_size, vocab_size, max_length).to(args['device'])
#################################

## Sequence-to-sequence models (Seq2Seq)

Modelos Sequence-to-Sequence (Seq2Seq) partem do mesmo princípio do Image Captioning, porém a entrada também é sequencial, de modo que a codificação também é realizada por um modelo recorrente. 

A atividade de hoje é no contexto de Neural Machine Translation (NMT), cujo pipeline é representado de forma simplificada a seguir. Note que o idioma source (francês) necessita apenas do token de finalização de sentença, enquanto o idioma target precisa de ambos os inicializadores e os finalizadores (```<sos>```, ```<eos>```), visto que a entrada da rede precisa do token de inicialização, mas a saída, através da qual será calculada a loss, é produzida apenas com a finalização.

![](https://pytorch.org/tutorials/_images/seq2seq.png)

Imagem retirada do tutorial de NMT do Pytorch: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

Encontre modelos de linguagem pré-treinados e arquiteturas implementadas em: http://opennmt.net/


In [0]:
# Baixando Dataset
!wget https://www.dropbox.com/s/gq36ksk347d36ln/translation_data.zip
!unzip translation_data.zip

--2019-08-04 14:29:17--  https://www.dropbox.com/s/gq36ksk347d36ln/translation_data.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/gq36ksk347d36ln/translation_data.zip [following]
--2019-08-04 14:29:17--  https://www.dropbox.com/s/raw/gq36ksk347d36ln/translation_data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc67850fb7241dcbf7a43578f5d7.dl.dropboxusercontent.com/cd/0/inline/Al8iFFgogPo_KxLMaaiD-EtddFZqTv7OjfIzlC16Wgi63CH-yiBJBn-SRUYGuLFxwtLbj4WnbFsKOAJDvRK0hHaAtS5yMWtWQbdnBedcrAL5WhTBJNMsaW1wTGT1cHZjGLQ/file# [following]
--2019-08-04 14:29:17--  https://uc67850fb7241dcbf7a43578f5d7.dl.dropboxusercontent.com/cd/0/inline/Al8iFFgogPo_KxLMaaiD-EtddFZqTv7OjfIzlC16Wgi63CH-yiBJBn-SRUYGuLFxwtLbj4WnbFsKOAJDvRK0hHaAtS

In [0]:
# Criando CSV de treino e teste para carregar com o TabularDataset

translation_path = 'data/eng-fra.txt'

samples = open(translation_path).read().split('\n')
  
# Write txt to csv
lines = (line.split("\t") for line in samples)
with open('translation_data.csv', 'w') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(('English', 'French'))
    writer.writerows(lines)
    
df = pd.read_csv('translation_data.csv')

# Reducing data (throwing out samples)
train, _ = train_test_split(df, test_size=0.6)

# Split train and test set 
train, test = train_test_split(train, test_size=0.02)

train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)


df = pd.read_csv('test.csv')
df.tail()

,English,French
1082,Are you sure you're warm enough?,Êtes-vous sûre d'avoir assez chaud ?
1083,Sea turtles have a long lifespan.,Les tortues marines ont une haute espérance de...
1084,What prevented him from coming?,Qu'est-ce qui l'a empêché de venir ?
1085,You will need a bodyguard.,Il te faudra une garde du corps.
1086,He will be delighted to see you.,Il sera ravi de vous voir.


In [0]:
# Preparação do dataset:
# Tokenização e inclusão dos tokens especiais (<eos>, <sos>, <pad>, <unk>)
TEXT_FR = data.Field(tokenize = 'spacy', include_lengths=True, eos_token = "<eos>")
TEXT_EN = data.Field(tokenize = 'spacy', include_lengths=True, init_token = "<sos>", eos_token = "<eos>")

fields = [('text_en', TEXT_EN), ('text_fr', TEXT_FR)]
train_data, test_data = data.TabularDataset.splits(
                                  path = '.',
                                  train = 'train.csv',
                                  test = 'test.csv',
                                  format = 'csv',
                                  fields = fields,
                                  skip_header = True)

for sample in train_data:
  print(sample.text_fr)
  print(sample.text_en)
  break
  
print(len(train_data), len(test_data))

['Ma', 'sœur', "m'a", 'demandé', 'de', 'lui', 'prêter', 'le', 'dictionnaire', '.']
['My', 'sister', 'asked', 'me', 'to', 'lend', 'her', 'the', 'dictionary', '.']
53250 1087


In [0]:
# Criando vocabulário
MAX_VOCAB_SIZE = 25000

TEXT_EN.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE)

TEXT_FR.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE)


# Instanciando bucket iterator
# Note que a ordenação é definida pelo 
# comprimento do par ** (en, fr) **
# Precisaremos empacotar ambas as sequências 
# para realizar o forward encapsulado.
train_iterator = data.BucketIterator(
    train_data, 
    batch_size = args['batch_size'],
    sort_key = lambda x:(len(x.text_fr), len(x.text_en)),
    sort_within_batch = True,
    device = args['device'])


test_iterator = data.BucketIterator(
    test_data, 
    batch_size = 1,
    sort_within_batch = False,
    device = args['device'])

for k, batch in enumerate(train_iterator):
  text_fr, lengths_fr = batch.text_fr
  text_en, lengths_en = batch.text_en
  
  print(text_fr)
  print(text_en)
  print(lengths_fr)
  print(lengths_en)
  break


tensor([[ 1237,    22,   178,    22,    15,    33,    22,   212,    94,     6],
        [ 1577,    12,   285,   739, 13315,    58,    20,  3545,    12,    10],
        [   29,   114,  1566,     7,   173,    49,    80,   100,    10,   127],
        [  995,     7,    16,   273, 13922,    36,     9,  4450, 16412,    80],
        [   16,    10,    23,     4,     4,   151,    10,   214,     5,     9],
        [  909,   462,    83,   419, 15317,   604,   424,    14,    10,   142],
        [ 6880,    25,   120,   246,    39,   213,   941,  2072,     7,    25],
        [   29,   483,    11,    82,    14,    14,   210,     4,   141,   403],
        [  907,   171,    90,   661,  1220,  2170,  1818,   708,  1044,   473],
        [    3,     3,     3,     3,     3,     3,     3,     3,     8,     3],
        [    2,     2,     2,     2,     2,     2,     2,     2,     2,     2]],
       device='cuda:0')
tensor([[   2,    2,    2,    2,    2,    2,    2,    2,    2,    2],
        [ 572,   19,    5

# Atividade Prática

Implemente a arquitetura apresentada na figura abaixo, criando **indivudalmente** os blocos encoder e decoder. Para o nosso exercício, eles não serão agrupados em uma única classe.

### EncoderRNN

Implemente a classe **EncoderRNN** composta de um passo de representação de palavras e um passo de caracterização de sequência, ou seja, implemente as seguintes camadas:
*  Embedding: como não usaremos vetores pré-treinados, a dimensão de saída dessa camada é um hiperparâmetro livre. Sugestão de tamanho: ```100```. Sua entrada é definida pelo tamanho do dicionário do idioma source (nesse caso o francês).
*  Dropout: ```0.1``` <br><br>
*  GRU: Defina ```hidden_size = 128``` para o encoder

### DecoderRNN

Implemente a classe **DecoderRNN**. Novamente é necessário uma camada de representação de palavras, seguida de uma camada de caracterização de sequências. Além disso, o decoder também deve possuir uma camada Linear de classificação, que transformará a representação de cada timestep (saída da RNN) em uma predição da próxima palavra.

* Embedding: A entrada definida pelo vocabulário do idioma target (inglês), saída é um hiperparâmetro livre (sugestão: ```100```).
* Dropout: ```0.1``` <br><br>
* GRU: Seus hiperparâmetros são inferíveis a partir das outras informações. **Lembre-se que a inicialização do hidden state é dada pelo último hidden state do encoder** (veja na função train). <br><br>
* Linear: Parâmetros inferíveis pelas outras informações. Quantas classes tem a predição de palavras em inglês?
* LogSoftmax: ativação da classificação.

No decoder, **implemente ambos os forward** para treinamento (encapsulado em batches) e para inferência (loop explícito sem targets).

<img src="https://drive.google.com/uc?export=view&id=1j8aLVymyvhGtM0lpfON0aDyPvUf4700U" width="850">

In [0]:
class EncoderRNN(nn.Module):
    # TODO: Implemente o encoder
      
      
class DecoderRNN(nn.Module):
    # TODO: Implemente o decoder
        
      

# TODO: Instancie ambos encoder e decoder


In [0]:
# Setting optimizer.
# encoder_optimizer = ...
# decoder_optimizer = ...

# Setting loss.
# NLLLoss usada em par com a ativação LogSoftmax
criterion = nn.NLLLoss().to(args['device'])


In [0]:
def train(train_loader, criterion, epoch):

    tic = time.time()
    
    # Setting network for training mode.
    encoder.train()
    decoder.train()

    # Lists for losses and metrics.
    train_loss = []
    
    # Iterating over batches.
    for i, batch_data in enumerate(train_loader):

        # Obtaining images, labels and paths for batch.
        text, text_lengths = batch_data.text_fr
        labs, labs_lengths = batch_data.text_en
        
        # Ignorando batches não ordenados para acelerar o treinamento
        if sorted(labs_lengths, reverse=True) != list(labs_lengths.data):
          continue

        # Clears the gradients of optimizer.
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # Forwarding.
        enc, enc_hidden = encoder(text, text_lengths)
        outs = decoder(enc_hidden, labs[:-1], labs_lengths-1)
        
        # Computing loss.
        loss = 0.
        for k, out in enumerate(outs):
          loss += criterion(out, labs[k+1])
        loss = loss.mean()
        
        # Computing backpropagation.
        loss.backward()
        
        # Weight update
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        # Updating lists.
        train_loss.append(loss.data.item())
    
    toc = time.time()
    
    train_loss = np.asarray(train_loss)
    
    # Printing training epoch loss and metrics.
    print('--------------------------------------------------------------------')
    print('[epoch %d], [train loss %.4f +/- %.4f], [training time %.2f]' % (
        epoch, train_loss.mean(), train_loss.std(), (toc - tic)))
    print('--------------------------------------------------------------------')

def test(test_loader, criterion, epoch):

    tic = time.time()
    
    # Setting network for evaluation mode (not computing gradients).
    encoder.eval()
    decoder.eval()

    # Lists for losses and metrics.
    test_loss = []
    
    print('********************************************************************')
    # Iterating over batches.
    for i, batch_data in enumerate(test_loader):

        # Obtaining images, labels and paths for batch.
        text, text_lengths = batch_data.text_fr
        labs, labs_lengths = batch_data.text_en
        
        
        # Forwarding.
        enc, enc_hidden  = encoder(text, text_lengths)
        outs = decoder(enc_hidden)

        if i < 2:
          print('Input:',  [TEXT_FR.vocab.itos[t] for t in text])
          print('Label:',  [TEXT_EN.vocab.itos[t] for t in labs[1:]])
          print('Output:', [TEXT_EN.vocab.itos[np.argmax(t.cpu().data)] for t in outs], '\n')
        
        
        # Computing approximate loss 
        labs = labs[1:]
        minlen = min(len(labs), len(outs))
        
        loss = 0.
        for k in range(minlen):
          loss += criterion(outs[k], labs[k])
        loss = loss.mean()
                
        # Updating lists.
        test_loss.append(loss.data.item())
    
    toc = time.time()

    test_loss = np.asarray(test_loss)
    
    # Printing training epoch loss and metrics.
   
    print('[epoch %d], [test loss %.4f +/- %.4f], [testing time %.2f]' % (
        epoch, test_loss.mean(), test_loss.std(), (toc - tic)))
    print('********************************************************************')


In [0]:
# Iterating over epochs.
for epoch in range(1, args['epoch_num'] + 1):

    # Training function.
    train(train_iterator, criterion, epoch)

    # Computing test loss and metrics.
    test(test_iterator, criterion, epoch)



--------------------------------------------------------------------
[epoch 1], [train loss 31.6066 +/- 13.2043], [training time 60.28]
--------------------------------------------------------------------
********************************************************************
Input: ['Je', 'conduirai', '.', '<eos>']
Label: ['I', "'ll", 'drive', '.', '<eos>']
Output: ['I', "'m", 'going', 'to', '.', '<eos>'] 

Input: ["J'aime", 'prendre', 'mon', 'café', 'sans', 'sucre', '.', '<eos>']
Label: ['I', 'like', 'my', 'coffee', 'without', 'sugar', '.', '<eos>']
Output: ['I', 'like', 'my', 'car', 'to', 'go', '.', '<eos>'] 

[epoch 1], [test loss 45.5876 +/- 22.4559], [testing time 8.31]
********************************************************************
--------------------------------------------------------------------
[epoch 2], [train loss 24.3321 +/- 10.1345], [training time 60.38]
--------------------------------------------------------------------
********************************************

KeyboardInterrupt: ignored